In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import optuna

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss

from sklearn.ensemble import RandomForestClassifier

import catboost
import xgboost as xgb
from xgboost.callback import EarlyStopping
import lightgbm as lgbm

import os
import sys
import datetime

import wandb

import warnings; warnings.filterwarnings('ignore')

sys.path.append('..')
from config import CFG
CFG = CFG()
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', CFG.NCOLS)
pd.set_option('display.max_rows', CFG.NROWS)


In [2]:
from colorama import Style, Fore
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
mgt = Style.BRIGHT + Fore.MAGENTA
grn = Style.BRIGHT + Fore.GREEN
gld = Style.BRIGHT + Fore.YELLOW
res = Style.RESET_ALL

rc = {
    "axes.facecolor": "#FFFEF8",
    "figure.facecolor": "#FFFEF8",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}
sns.set(rc=rc)
palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

In [3]:
orig = pd.read_csv(os.path.join(CFG.RAW_DATA, 'Pulsar.csv'))
train = pd.read_csv(os.path.join(CFG.RAW_DATA, 'train.csv')).drop(columns='id')
test = pd.read_csv(os.path.join(CFG.RAW_DATA, 'test.csv')).drop(columns='id')

In [11]:
with open('data\\features.txt', 'r') as f:
    features = f.readlines()
features = features[0].split(' ')
features = features[0].split(' ')
features

['SD SD_DMSNR_Curve Skewness_Power3 EK EK_Power3 cos(EK) SD_multiply_EK cos(EK)_multiply_SD SD_DMSNR_Curve_multiply_Skewness_Power3 EK_divide_EK_Power3 EK_multiply_SD EK_divide_SD EK_multiply_SD_DMSNR_Curve SD_DMSNR_Curve_divide_SD SD_multiply_EK_Power3 SD_DMSNR_Curve_multiply_SD EK_Power3_multiply_SD_DMSNR_Curve cos(EK)_multiply_SD_DMSNR_Curve ']

In [4]:
sdek = train['SD'] * train['EK']
eksd = train['EK'] * train['SD']

In [7]:
# def f_eng(df):
#     df['Skewness_Power3'] = df['Skewness'].apply(lambda x: x**3)
#     df['EK_Power3'] = df['EK'].apply(lambda x: x**3)
#     df['cos(EK)'] = df['EK'].apply(lambda x: np.cos(x))
#     df['SD_x_EK'] = df['SD'] * df['EK']
#     df['cos(EK)_x_SD'] = df['cos(EK)'] * df['SD']
#     df['SD_DMSNR_Curve_x_Skewness_Power3'] = df['SD_DMSNR_Curve'] * df['Skewness_Power3']
#     df['EK_divide_EK_Power3'] = df['EK'] / df['EK_Power3']
#     df['EK_multiply_SD'] = df['EK'] * df['SD']
#     df['EK_divide_SD'] = df['EK'] / df['SD']
#     df['EK_multiply_SD_DMSNR_Curve'] = df['EK'] * df['SD_DMSNR_Curve']
#     df['SD_DMSNR_Curve_divide_SD'] = df['SD_DMSNR_Curve'] / df['SD']
#     df['SD_multiply_EK_Power3'] = df['SD'] * df['EK_Power3']
#     df['SD_DMSNR_Curve_multiply_SD'] = df['SD_DMSNR_Curve'] * df['SD']
#     df['EK_Power3_multiply_SD_DMSNR_Curve'] = df['EK_Power3'] * df['SD_DMSNR_Curve']
#     df['cos(EK)_multiply_SD_DMSNR_Curve'] = df['cos(EK)'] * df['SD_DMSNR_Curve']
#     return df

In [8]:
train = f_eng(train)
test = f_eng(test)
orig = f_eng(orig)

In [7]:
y = train['Class']
X = train.drop(['Class'], axis=1)

In [40]:
PATIENCE = 100

modelsCB = []
predsCB = []

k_fold = RepeatedStratifiedKFold(n_splits=CFG.NFOLDS, n_repeats=CFG.REPEATS, random_state=CFG.SEED) 

param_list = [
    'iterations', 'sampling_frequency', 'leaf_estimation_method', 'grow_policy', 'bayesian_matrix_reg', 'l2_leaf_reg', 'random_strength', 'model_size_reg',
    'od_wait', 'random_seed', 'depth', 'posterior_sampling', 'border_count', 'min_data_in_leaf', 'learning_rate', 'leaf_estimation_iterations', 'bootstrap_type', 'max_leaves'
]
param_dict = {param: [] for param in param_list}
param_dict['iterations'] = 9999
param_dict['random_state'] = CFG.SEED
param_dict['learning_rate'] = 0.15

wandb.init(project='S3E10', tags='catboost', config=param_dict, group='catboost')

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = catboost.CatBoostClassifier(iterations=param_dict['iterations'], random_state=param_dict['random_state'], learning_rate=param_dict['learning_rate'])
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds = PATIENCE,
          verbose=150,
         )
    
    params = model.get_all_params()
    del params['nan_mode']; del params['od_pval']; del params['penalties_coefficient']; del params['boosting_type']; del params['score_function']; del params['task_type']; del params['model_shrink_rate']; del params['best_model_min_trees']; del params['leaf_estimation_backtracking']; del params['sparse_features_conflict_fraction']; del params['classes_count']; del params['auto_class_weights']; del params['class_names']; del params['pool_metainfo_options']; del params['boost_from_average']; del params['rsm']; del params['od_type']; del params['force_unit_auto_pair_weights']; del params['eval_fraction']; del params['feature_border_type']; del params['model_shrink_mode']; del params['loss_function']; del params['eval_metric']; del params['use_best_model']
    wandb.log({
        'val_logloss': model.best_score_['validation']['Logloss'],
        'train_logloss': model.best_score_['learn']['Logloss']})
    modelsCB.append(model)
    predsCB.append(model.predict_proba(test)[:, 1])
wandb.config.update(params)
wandb.finish()

0:	learn: 0.3571962	test: 0.3576363	best: 0.3576363 (0)	total: 10.2ms	remaining: 1m 41s
150:	learn: 0.0247359	test: 0.0297552	best: 0.0297552 (150)	total: 1.07s	remaining: 1m 9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.0297551636
bestIteration = 150

Shrink model to first 151 iterations.
0:	learn: 0.3567845	test: 0.3572217	best: 0.3572217 (0)	total: 7.42ms	remaining: 1m 14s
150:	learn: 0.0234098	test: 0.0319930	best: 0.0319773 (122)	total: 945ms	remaining: 1m 1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.03195111118
bestIteration = 158

Shrink model to first 159 iterations.
0:	learn: 0.3560314	test: 0.3556622	best: 0.3556622 (0)	total: 6.89ms	remaining: 1m 8s
150:	learn: 0.0226332	test: 0.0336651	best: 0.0333159 (73)	total: 887ms	remaining: 57.9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.03331586375
bestIteration = 73

Shrink model to first 74 iterations.
0:	learn: 0.3482127	test: 0.3482768	best: 0.3482768

train_logloss,▆▄▇█▅▆█▁▅
val_logloss,▁▅█▂▅█▃▇▃
train_logloss,0.02063
val_logloss,0.03077


In [69]:
def objective(trial):
    depth = trial.suggest_int('depth', 2, 7)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1, 100)
    random_strength = trial.suggest_loguniform('random_strength', 1, 100)
    max_bin = trial.suggest_int('max_bin', 2, 255)
    od_wait = trial.suggest_int('od_wait', 10, 100)
    grow_policy = trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    
    params = {
        'depth': depth,
        'learning_rate': learning_rate,
        'l2_leaf_reg': l2_leaf_reg,
        'random_strength': random_strength,
        'max_bin': max_bin,
        'od_wait': od_wait,
        'grow_policy': grow_policy,
        }
    predsCB = []
    k_fold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=CFG.SEED) 
    for train_index, test_index in k_fold.split(X, y):
        X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        
        model = catboost.CatBoostClassifier(**params)
        
        model.fit(X=X_train, y=y_train,
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds = PATIENCE,
            verbose=False,
            )
        
        predsCB.append(model.predict_proba(X)[:, 1])
    resCB = np.mean(predsCB, axis=0)
    
    return log_loss(y, resCB)

study = optuna.create_study(direction='minimize')

[I 2023-03-15 11:04:08,327] A new study created in memory with name: no-name-d78e3ca7-f05c-4221-b72a-d0e28ec3cc29


In [70]:
study.optimize(objective, n_trials=50)
CB_params = study.best_params
CB_params

[I 2023-03-15 11:07:00,194] Trial 0 finished with value: 0.030981873846558164 and parameters: {'depth': 3, 'learning_rate': 0.0118458460425657, 'l2_leaf_reg': 1.5787606451765093, 'random_strength': 1.7163168926352992, 'max_bin': 35, 'od_wait': 33, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.030981873846558164.
[I 2023-03-15 11:07:20,716] Trial 1 finished with value: 0.02615298628834511 and parameters: {'depth': 4, 'learning_rate': 0.26214654564289275, 'l2_leaf_reg': 17.23264243955274, 'random_strength': 2.1418484394453747, 'max_bin': 204, 'od_wait': 100, 'grow_policy': 'Depthwise'}. Best is trial 1 with value: 0.02615298628834511.
[I 2023-03-15 11:07:52,874] Trial 2 finished with value: 0.026203177812139253 and parameters: {'depth': 7, 'learning_rate': 0.28973035911294737, 'l2_leaf_reg': 28.36000050590904, 'random_strength': 51.48001898887693, 'max_bin': 114, 'od_wait': 74, 'grow_policy': 'Lossguide'}. Best is trial 1 with value: 0.02615298628834511.
[W 2023-03-15 11:09:

KeyboardInterrupt: 

In [22]:
def get_fi(model):
    feature_importance =  [model[x].feature_importances_ for x in range(CFG.NFOLDS*CFG.REPEATS)]
    feature_importance = np.average(feature_importance,axis=0)
    feature_df = pd.DataFrame(feature_importance, index=X.columns)

    fig, ax = plt.subplots(figsize=(12, 4))
    sns.barplot(x=feature_df.values.squeeze(), y=feature_df.index,
                color=palette[-3], linestyle="-", width=0.5, errorbar='sd',
                linewidth=0.5, edgecolor="black", ax=ax)
    ax.set_title('Feature Importance', fontdict={'fontweight': 'bold'})
    ax.set(xlabel=None)

    for s in ['right', 'top']:
        ax.spines[s].set_visible(False)

In [ ]:
get_fi(modelCB)

In [41]:
from xgboost import XGBClassifier, XGBRegressor
from wandb.xgboost import WandbCallback
modelsXB = []
predsXB = []

PATIENCE = 50

wandb.init(project='S3E10', tags='xgboost', group='xgboost', reinit=True)

xgbr_params = {
            'n_estimators':9999,
            # 'max_depth': 4,
            # 'learning_rate': 0.05333221939055333,
            # 'min_child_weight': 4,
            # 'gamma': 5.301218558776368e-08,
            # 'subsample': 0.41010429946197946,
            # 'colsample_bytree': 0.8298539920447499,
            # 'reg_alpha': 0.000517878113716743,
            # 'reg_lambda': 0.00030121415155097723,
            'n_jobs': -1,
            'objective': 'binary:logistic',
            'verbosity': 0,
            'eval_metric': 'logloss',
            'random_state': CFG.SEED}

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = XGBClassifier(**xgbr_params)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds = PATIENCE,
          verbose = 100,
          callbacks=[WandbCallback()]
         )
    modelsXB.append(model)
    predsXB.append(model.predict_proba(test)[:, 1])
wandb.finish()

[0]	validation_0-logloss:0.44527
[80]	validation_0-logloss:0.03082
[0]	validation_0-logloss:0.44559
[72]	validation_0-logloss:0.03333
[0]	validation_0-logloss:0.44572
[73]	validation_0-logloss:0.03652
[0]	validation_0-logloss:0.44533
[82]	validation_0-logloss:0.03163
[0]	validation_0-logloss:0.44553
[76]	validation_0-logloss:0.03415
[0]	validation_0-logloss:0.44557
[73]	validation_0-logloss:0.03487
[0]	validation_0-logloss:0.44539
[76]	validation_0-logloss:0.03217
[0]	validation_0-logloss:0.44569
[73]	validation_0-logloss:0.03486
[0]	validation_0-logloss:0.44538
[72]	validation_0-logloss:0.03338


best_iteration,▇▁▂█▄▂▄▂▁
best_score,▁▄█▂▅▆▃▆▄
epoch,▂▃▅▆█▂▃▅▇▂▃▅▆▁▃▄▆▇▁▃▅▆█▂▄▅▇▂▃▅▆█▂▅▆█▂▃▅▇
validation_0-logloss,▂▁▁▁▁▂▁▁▁▂▁▁▁▃▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,22
best_score,0.03227
epoch,72


In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 10000)
    max_depth = trial.suggest_int('max_depth', 2, 7)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    gamma = trial.suggest_loguniform('gamma', 1e-8, 1.0)
    subsample = trial.suggest_discrete_uniform('subsample', 0.4, 1.0, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.4, 1.0, 0.1)
    reg_alpha = trial.suggest_loguniform('reg_alpha', 1e-8, 10.0)
    reg_lambda = trial.suggest_loguniform('reg_lambda', 1e-8, 10.0)

    
    k_fold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=CFG.SEED) 
    preds_eval = []
    for train_index, test_index in k_fold.split(X, y):
        X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        
        model = XGBClassifier(params)
        
        model.fit(X=X_train, y=y_train,
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds = PATIENCE,
            verbose = False
            )
        
        predsXB.append(model.predict_proba(X)[:, 1])
    resXB = np.mean(predsXB, axis=0)
    
    return log_loss(y, resXB)

study = optuna.create_study(direction='minimize')

In [42]:
get_fi(modelsXB)

NameError: name 'get_fi' is not defined

In [43]:
from wandb.lightgbm import wandb_callback, log_summary

wandb.init(project='S3E10', tags='lightgbm', group='lightgbm', reinit=True)

modelsLB = []
predsLB = []

PATIENCE = 50

# gpu_params = {'device' : "gpu"}
lgbr_params = {
            'n_estimators': 9999,
            # 'max_depth': 5,
            'learning_rate': 0.00693702575527996,
            # 'subsample': 0.20851841295589477,
            # 'colsample_bytree': 0.5784778854092203,
            # 'reg_alpha': 0.2622912287429849,
            # 'reg_lambda': 2.8702494234117617e-08,
            'objective': 'binary',
            'metric': 'logloss',
            'boosting_type': 'gbdt',
           
            'random_state': CFG.SEED
        }

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = lgbm.LGBMClassifier(**lgbr_params)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          eval_metric = 'logloss',
          early_stopping_rounds = PATIENCE,
          verbose = 150,
          callbacks=[wandb_callback()]
         )
    modelsLB.append(model)
    predsLB.append(model.predict_proba(test)[:, 1])

[150]	valid_0's binary_logloss: 0.0859365
[300]	valid_0's binary_logloss: 0.0467715
[450]	valid_0's binary_logloss: 0.0350023
[600]	valid_0's binary_logloss: 0.0314534
[750]	valid_0's binary_logloss: 0.0303514
[900]	valid_0's binary_logloss: 0.0299974
[1050]	valid_0's binary_logloss: 0.0298667
[150]	valid_0's binary_logloss: 0.0870119
[300]	valid_0's binary_logloss: 0.0482203
[450]	valid_0's binary_logloss: 0.0369208
[600]	valid_0's binary_logloss: 0.033632
[750]	valid_0's binary_logloss: 0.0326962
[900]	valid_0's binary_logloss: 0.0324952
[1050]	valid_0's binary_logloss: 0.0323964
[150]	valid_0's binary_logloss: 0.0872709
[300]	valid_0's binary_logloss: 0.0489055
[450]	valid_0's binary_logloss: 0.0379273
[600]	valid_0's binary_logloss: 0.0350015
[750]	valid_0's binary_logloss: 0.034133
[900]	valid_0's binary_logloss: 0.0340224
[150]	valid_0's binary_logloss: 0.0858954
[300]	valid_0's binary_logloss: 0.0468565
[450]	valid_0's binary_logloss: 0.0353913
[600]	valid_0's binary_logloss: 0.

In [ ]:
get_fi(modelsLB)

In [51]:
wandb.init(project='S3E10', tags='randomforest', group='randomforest', reinit=True)
modelsRF = []
predsRF = []

params = {
        'criterion': 'log_loss',
        'n_jobs': -1,
        'random_state': CFG.SEED,
        'verbose': 1
    }

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = RandomForestClassifier(**params)
    model.fit(X=X_train, y=y_train)
    yprobas = model.predict_proba(X_valid)
    ypred = model.predict(X_valid)
    score = log_loss(y_valid, ypred)
    print(f'Score: {score}')
    
    modelsRF.append(model)
    predsRF.append(model.predict_proba(test)[:, 1])
    wandb.sklearn.plot_classifier(model, 
                              X_train, X_valid, 
                              y_train, y_valid,
                              ypred, yprobas,
                              [0, 1],
                              is_binary=True, 
                              model_name='RandomForest')

wandb.finish()

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.6s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
wandb: 
wandb: Plotting RandomForest.


Score: 0.3219168798150202


wandb: Logged feature importances.
wandb: Logged confusion matrix.
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
wandb: Logged summary metrics.
wandb: Logged class proportions.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using b

Score: 0.3357133175213782


wandb: Logged feature importances.
wandb: Logged confusion matrix.
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
wandb: Logged summary metrics.
wandb: Logged class proportions.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  85 out of 100 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1

...

wandb: Logged feature importances.
wandb: Logged confusion matrix.
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
wandb: Logged summary metrics.
wandb: Logged class proportions.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  85 out of 100 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1

Score: 0.3412318926039214


wandb: Logged feature importances.
wandb: Logged confusion matrix.
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
wandb: Logged summary metrics.
wandb: Logged class proportions.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  85 out of 100 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  85 out of 100 | elapsed:    0.1s remaining:    0.0s


Score: 0.32375640484253465


wandb: Logged feature importances.
wandb: Logged confusion matrix.
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
wandb: Logged summary metrics.
wandb: Logged class proportions.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using b

In [52]:
def coef_objective(trial):
    a = trial.suggest_float('a', 0, 0.2)
    b = trial.suggest_float('b', 0.5, 1)
    c = trial.suggest_float('c', 0.5, 1)
    e = trial.suggest_float('e', 0, 1)
    
    preds_eval = []
    for model in modelsCB:
        preds_eval.append(model.predict_proba(X)[:, 1])
    resCB = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsXB:
        preds_eval.append(model.predict_proba(X)[:, 1])
    resXB = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsLB:
        preds_eval.append(model.predict_proba(X)[:, 1])
    resLB = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsRF:
        preds_eval.append(model.predict_proba(X)[:, 1])
    resRF = np.average(np.array(preds_eval),axis=0)
    
    res1 =  (resCB * a + resXB * b + resLB * c + resRF * e)/(a + b + c + e)
    res = log_loss(y, res1)
    return res

study = optuna.create_study(direction= 'minimize')

[I 2023-03-15 10:24:17,832] A new study created in memory with name: no-name-33a1195a-b1e1-4ef8-b1d4-60a9b4eefb16


In [53]:
study.optimize(coef_objective, n_trials=100)
coef_dict = study.best_params
coef_dict

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_job

{'a': 0.01423510666910206,
 'b': 0.5829895527851879,
 'c': 0.5377258076877631,
 'e': 0.9793927248236828}

In [55]:
predCB = np.average(np.array(predsCB),axis=0)
predXB = np.average(np.array(predsXB),axis=0)
predLB = np.average(np.array(predsLB),axis=0)
predRF = np.average(np.array(predsRF),axis=0)

pred = (predCB * coef_dict['a'] + predXB * coef_dict['b'] + predLB * coef_dict['c'] + predRF * coef_dict['e']) / (coef_dict['a'] + coef_dict['b'] + coef_dict['c'] + coef_dict['e'])

In [56]:
submission = pd.read_csv(os.path.join(CFG.DATA_PATH, 'sample_submission.csv'))
submission['Class'] = pred
submission.to_csv(os.path.join(CFG.BASE_PATH, 'submissions', 'CT-XG-LGB-RF_poly+trig-features_v1.csv'), index=False)